# Pokemon Image Classifier

Image classifier built using TensorFlow

## Usage

To identify what species of Pokemon a given image resembles. Could be used to see which Pokemon one's pet or drawing most closely resembles

## Data

I combined two datasets from Kaggle and modified the datasets such that there includes an image repository and a csv file of pokemon names and their corresponding image file names.

The data sets used:
* https://www.kaggle.com/datasets/vishalsubbiah/pokemon-images-and-types
* https://www.kaggle.com/datasets/lantian773030/pokemonclassification

## Imports and setup

In [3]:
import tensorflow as tf
import tensorflow_hub as hub 
print(tf.__version__)

2.12.0


In [4]:
# check GPU (Google Colab)
print("GPU available") if tf.config.list_physical_devices("GPU") else "GPU not available"

GPU available
